# Entry 23: Egonet Density

The next metric on our list to translate from the global graph to a local neighborhood is density. For a high level discussion of density, see [Entry G7](https://julielinx.github.io/blog/g07_global_density_diameter/) and [Entry G15](https://julielinx.github.io/blog/g15_global_density_comparison/). G15 in particulare includes picture examples of dense vs sparse graphs.

There is some variation in the information needed between unimodal and bimodal graph models. For unimodal models, all you need is a total count of nodes.

When calculating density for bimodal models, you need separate counts for heroes and comics. This is due to the equation to calculate the number of possible relationships. This makes the query for the bimodal models slightly longer.

I solve this challenge by using two queries, one for unimodal and one for bimodal.

I only run densities out to next nearest neighbors (2 steps) because the "average node" is connected to all the nodes it can be connected to with 2.6 steps. So if we go out 3 steps, we may as well run the density of the full graph which defeats the purpose of a local metric.

## The Problem

As discussed in [Entry G18](https://julielinx.github.io/blog/g18_egocentric_networks/) we often want to know about specific nodes, or have information that will help us differentiate between nodes to find things of interest.

The best place to start is the immediate neighbors and the next nearest neighbors. If you remember from [Entry G10](https://julielinx.github.io/blog/g10_local_metrics/) we need to find outliers for closer inspection. In the Panama Papers and other ICIJ investigations we can use these outliers to locate entities for closer investigation to see whether they're acting in bad good or bad faith.

## The Solution

We already have all the pieces we need in order to calculate local density, we just need to put them together.

- Determine step level
- Isolate a subgraph
- Get the node and relationship counts
- Calculate the number of possible relationships
- Calculate density

### Step Level

As indicated above, we want both the nearest neighbors (one step) and next nearest neighbors (two steps). For ease of reference and to keep the calculations comparable across graph models, I define one step in the bimodal models as Hero to Hero, the same as in the unimodal model.

I also ran the calculations on both the graph as a whole, as well as just for villain neighbors.

The logic of this is that bad actors tend to be connected to other bad actors in some way. For example, auto fraud rings in California are known to have networks: the same drivers/passengers are involved in many accidents, the same set of law firms represent those drivers/passengers, and the same clinics treat the alleged injuries of those drivers/passengers.

We should be able to see these networks and densities in the graph. Using just villian neighbors will hopefully give a clearer signal that including all contacts. After all, even legitimately injured people probably attend the clinics that are involved in auto fraud.

The code is in the following notebooks:
- Nearest Neighbors: [Entry 23a notebook](https://github.com/julielinx/datascience_diaries/blob/master/graph/23a_nb_%20egonet_densities.ipynb)
- Next Nearest Neighbors [Entry 23b notebook](https://github.com/julielinx/datascience_diaries/blob/master/graph/23b_nb_egonet_densities.ipynb)
- Nearest Villain Neighbors: [Entry 23c notebook](https://github.com/julielinx/datascience_diaries/blob/master/graph/23c_nb_%20egonet_densities.ipynb)
- Next Nearest Villain Neighbors: [Entry 23d notebook](https://github.com/julielinx/datascience_diaries/blob/master/graph/23d_nb_egonet_densities.ipynb)

### Isolate Subgraph

For the past several metrics, we've used `apoc.path.spanningTree`. As discussed in [Entry G19](https://julielinx.github.io/blog/g19_neighborhood_node_cts/#apocpathspanningtree), this is a breadth first search algorithm, which means it finds only the first/shortest path between nodes.

For this problem, we need all relationships that meet certain criteria. The best candidate to accomplish this task is the `apoc.path.subgraphAll` algorithm.

One of the limitations mentioned previously in [Entry G19](https://julielinx.github.io/blog/g19_neighborhood_node_cts/#apocpathsubgraphall) as the increasing time at larger step distances. Since we're only going out two steps, the distance penalty wasn't a problem.

The graph query to isolate the per node subgraph looks something like this (changes are introduced to account for nearest vs next nearest and all vs villain - see the code for the specifics):

```
MATCH (h:Hero)
call apoc.path.subgraphAll(h, {maxLevel: 2, relationshipFilter:'KNOWS'})
YIELD nodes, relationships
```

### Get Node and Relationship Counts

It was very easy to work with the lists of things when I needed straight counts of everything in the subgraph. The `size` function applied to each of the return parameters (`nodes` and `relationships`) in the `RETURN` gave me exactly what I needed.

```
RETURN h.name as hero, labels(h)[-1] as h_type, size(relationships) as rel_ct, size(nodes) as node_ct
```

### Calculate the Number of Possible Relationships

The full query from the above two steps looks like this:

```
MATCH (h:Hero)
call apoc.path.subgraphAll(h, {maxLevel: 2, relationshipFilter:'KNOWS'})
YIELD nodes, relationships
RETURN h.name as hero, labels(h)[-1] as h_type, size(relationships) as rel_ct, size(nodes) as node_ct
```

This is the resulting dataframe:

<img src='images/23_query_results.png'>

You may or may not recall the equation for the number of possible relationships from way back in [Entry G7](https://julielinx.github.io/blog/g07_global_density_diameter/). As discussed in that entry, the calcuation depends on the graph model (unimodal vs bimodal) and whether or not the graph is directed or undirected. As a refresher, here are the equations (for more details on the differences in the equations, please refer to Entry G7's [Number of possible relationships](https://julielinx.github.io/blog/g07_global_density_diameter/#number-of-possible-relationships) section and Entry G15's [Possible Relationships](https://julielinx.github.io/blog/g15_global_density_comparison/#possible-relationships) section):

#### Unimodal

Multiply the number of nodes by one less than the number of nodes.

**Directed unimodal graph:**

$pr = n (n - 1)$

**Undirected unimodal graph:**

$pr = \frac{n (n - 1)}{2}$

Where

- pr = number of possible relationships
- n = node count

#### Bimodal

**Directed bimodal graph:**

$pr = (n1 \times n2) \times 2$

**Undirected bimodal graph:**

$pr = n1 \times n2$

Where

- pr = number of possible relationships
- n1 = node type 1
- n2 = node type 2

For the unimodal calculation we only need the `node_ct` column.

For the bimodal calculation we need to separate out the count of nodes by their label: hero or comic. Because of this, the query and results for the bimodal models is slightly different from what's listed above. It looks like this:

<img src='images/23_bi_query_results.png'>

Notice that the `Hero` count matches the `node_ct` from the unimodal example. This is because the `Hero` nodes aren't in the unimodal graph, so these are all extra nodes.

### Calculate Density

Once we have the number of possible relationships, the density calculation is easy and standard across model types:

$d =\frac{r}{pr}$

Where:

- d = density
- r = relationship count
- pr = number of possible relationships

### Get the Answer

Now that we've got all the values, the final dataframe looks like this:

<img src='images/23_density_df.png'>

Remember, since this is a local density, we have a value for every node. As such, our next step is to get a handle on the distrubution of the values.

## The Fail

It wasn't until I went to visualize the distributions that I realized the `apoc.path.subgraphAll` algorithm returns all relationships for the nodes, not just the ones specified in the `relationshipFilter`. I can see how this may be helpful for visualizing the graph, but as for the actual returned results, the intuitive reational is that you only get back the relationships you specifically asked for.

There are no parameters that change the behavior to the one I assumed (return only the relationships asked for in `relationshipFilter`). To get this behavior required advanced Cypher syntax. For now, I just subtracted the `KNOWS` relationship counts and recalculated the density.

*Caution*, I left the incorrect values in the `.csv` files and only corrected the mixed graph model bimodal version in the actual code for the distrubution visualizations.

I go into the behavior of the algorithm in more detail in the next entry.

## Up Next

Local Density Distribution

## Resources

- [Entry G7: Density and Diameter](https://julielinx.github.io/blog/g07_global_density_diameter/)
- [Entry G10: Local Metrics](https://julielinx.github.io/blog/g10_local_metrics/)
- [Entry G15: Global Density Comparison](https://julielinx.github.io/blog/g15_global_density_comparison/)
- [Entry G18: Egocentric Networks](https://julielinx.github.io/blog/g18_egocentric_networks/)
- [Entry G19: Neighborhood Node Counts](https://julielinx.github.io/blog/g19_neighborhood_node_cts/#apocpathsubgraphall)